In [2]:
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [4]:
# Set options
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

data_raw = pd.read_csv("STAT_444_project_data_power_transformed.csv", low_memory = True)

#data_raw.dropna()
data_raw.reset_index(drop=True, inplace=True)
train_raw, test_raw = train_test_split(data_raw, test_size=0.1, random_state=42)
#train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
#train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
#test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)



df_train = pd.DataFrame(train_raw).drop(["date_1","date_2", "temp2_c", "temp2_min_c",
            "wind_speed50_ave_m_s"], axis=1)
df_test = pd.DataFrame(test_raw).drop(["date_1","date_2", "temp2_c", "temp2_min_c",
            "wind_speed50_ave_m_s"], axis=1)


len(df_train.columns)
df_train

,day,month,year,temp2_max_c,temp2_ave_c,surface_pressure_pa,wind_speed50_max_m_s,wind_speed50_min_m_s,prectotcorr,total_demand_mw,max_generation_mw
1013,11.211578,6.809483,-8.840693e-34,9173.644062,4.311080e+11,1.136752e+20,2.301639,0.972103,0.003981,38.796139,-0.158464
1432,12.589254,1.000000,-8.753636e-34,6130.530937,8.903655e+10,1.144632e+20,2.091786,1.164213,0.003981,39.147977,-0.159749
1715,2.143547,6.809483,-8.753636e-34,9072.726139,6.562704e+11,1.104622e+20,3.439450,1.366268,0.003981,42.414245,-0.153287
32,2.143547,1.741101,-8.928703e-34,5589.762571,5.917282e+10,1.120030e+20,1.851023,0.792186,0.003981,35.483341,-0.168962
869,16.801099,4.192963,-8.840693e-34,9541.061003,1.637914e+12,9.980018e+19,3.830037,1.620657,3.060046,40.241941,-0.156318
...,...,...,...,...,...,...,...,...,...,...,...
1130,9.849155,2.408225,-8.797046e-34,12795.958602,9.489104e+11,1.118923e+20,3.578425,0.857917,0.831224,40.832416,-0.154347
1294,26.985657,5.278032,-8.797046e-34,9970.881089,1.377687e+12,1.061646e+20,3.884985,1.641728,1.551119,41.452351,-0.153719
860,4.594793,4.192963,-8.840693e-34,11702.912265,1.980164e+12,1.038656e+20,3.398117,1.464875,2.696504,37.762534,-0.160545
1459,8.503698,1.741101,-8.753636e-34,3860.334426,9.837790e+09,1.158256e+20,2.545942,1.206570,0.251189,39.041613,-0.160198


In [5]:
# Neural Network
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm

In [6]:
df_z_scaled = df_train.copy()
df_tz = df_test.copy()
# apply normalization techniques 
for column in df_z_scaled.columns:
    df_z_scaled[column] = df_z_scaled[column].astype(float)
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / (df_z_scaled[column].std())

for column in df_tz.columns:
    df_tz[column] = df_tz[column].astype(float)
    df_tz[column] = (df_tz[column] -
                           df_tz[column].mean()) / (df_tz[column].std())
    


In [8]:
df_xtrain = df_z_scaled.drop(['total_demand_mw'], axis=1)
df_xtest = df_tz.drop(['total_demand_mw'], axis=1)
df_ytrain = df_z_scaled['total_demand_mw']
df_ytest = df_tz['total_demand_mw']


df_tensor_xtr = torch.tensor(df_xtrain.values, dtype=torch.float32)
df_tensor_xte = torch.tensor(df_xtest.values, dtype=torch.float32)
df_tensor_ytr = torch.tensor(df_ytrain.values, dtype=torch.float32)
df_tensor_yte = torch.tensor(df_ytest.values, dtype=torch.float32)
df_tensor_xtr = torch.nan_to_num(df_tensor_xtr)
df_tensor_xte = torch.nan_to_num(df_tensor_xte )
df_tensor_ytr = torch.nan_to_num(df_tensor_ytr)
df_tensor_yte = torch.nan_to_num(df_tensor_yte )


In [8]:
df_tensor_xtr

tensor([[-1.1131,  1.6514, -0.7596,  ..., -1.1180, -0.5416, -1.7044],
        [ 1.5039, -0.9312, -0.1149,  ...,  0.4594, -0.5416, -0.3560],
        [-0.5442, -1.5051,  1.1745,  ..., -0.6835, -0.4981, -0.4646],
        ...,
        [-0.6580, -0.3573, -0.1149,  ..., -0.9394,  0.2176, -0.1865],
        [ 1.6177,  1.6514,  0.5298,  ..., -0.4037, -0.5416, -0.7082],
        [ 1.7314, -1.5051,  0.5298,  ..., -0.0882, -0.5416, -0.7737]])

In [9]:
class dataset(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.x = df_tensor_xtr
        self.t = df_tensor_ytr
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]
    
    def inputs(self):
        return self.x
    
    def targets(self):
        return self.t

In [10]:
class datasett(torch.utils.data.Dataset):
    def __init__(self):
        super().__init__()
        self.x = df_tensor_xte
        self.t = df_tensor_yte
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]
    
    def inputs(self):
        return self.x
    
    def targets(self):
        return self.t

In [11]:
#batch_size_train = 256
#train_dl = torch.utils.data.DataLoader(dataset(), batch_size=batch_size_train, shuffle=True)
test_all = torch.utils.data.DataLoader(datasett(), batch_size=len(df_test), shuffle=True)

In [12]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [14]:
class RegressorNN(nn.Module):
    '''
     
    '''
    def __init__(self):
        self.losses = []
        super().__init__()
        
        #===== YOUR CODE HERE =====
        self.seq= nn.Sequential(
            nn.Linear(10, 32),
            nn.Sigmoid(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, 8),
            nn.Sigmoid(),
            nn.Linear(8, 1))
    
    def forward(self, x):
        #===== YOUR CODE HERE =====
        x = self.seq(x)
        return x  # replace this line

In [15]:
from sklearn.model_selection import KFold

In [16]:
loss_fn = nn.MSELoss()

k_folds = 5
num_epochs = 2000
# Set fixed random number seed
torch.manual_seed(42)


# Define the K-fold Cross Validator
kf = KFold(n_splits=k_folds, shuffle=True)

In [17]:
all_val_losses = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset())):
    print(f'Fold {fold+1}')
    
    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
    
    train_loader = torch.utils.data.DataLoader(dataset(), batch_size=256, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset(), batch_size=256, sampler=val_sampler)
    
    net1 = RegressorNN()
    if torch.cuda.is_available():
      net1 = net1.cuda()
    net1.apply(reset_weights)
    optimizer = torch.optim.SGD(net1.parameters(), lr=0.00001, momentum=0.8)
    
    for epoch in tqdm(range(num_epochs)):
        net1.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            predictions = net1(X_batch)
            loss = loss_fn(predictions, y_batch)
            loss.backward()
            optimizer.step()
    
    # Validation
    net1.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            predictions = net1(X_batch)
            loss = loss_fn(predictions, y_batch)
            val_losses.append(loss.item())
    fold_val_loss = np.mean(val_losses)
    all_val_losses.append(fold_val_loss)
    print(f'Validation Loss for fold {fold+1}: {np.mean(val_losses)}')


mean_cv_loss = np.mean(all_val_losses)
std_cv_loss = np.std(all_val_losses)

print(f'Final CV Loss: {mean_cv_loss}')

Fold 1
Reset trainable parameters of layer = Linear(in_features=10, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


  0%|          | 0/2000 [00:00<?, ?it/s]c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([83])) that is different to the input size (torch.Size([83, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 2000/2000 [00:57<00:00, 34.80it/s]
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([85])) t

Validation Loss for fold 1: 1.021453619003296
Fold 2
Reset trainable parameters of layer = Linear(in_features=10, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


100%|██████████| 2000/2000 [00:59<00:00, 33.45it/s]


Validation Loss for fold 2: 1.126146912574768
Fold 3
Reset trainable parameters of layer = Linear(in_features=10, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


...

Validation Loss for fold 3: 0.9771924018859863
Fold 4
Reset trainable parameters of layer = Linear(in_features=10, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


100%|██████████| 2000/2000 [01:05<00:00, 30.77it/s]


Validation Loss for fold 4: 1.0633456408977509
Fold 5
Reset trainable parameters of layer = Linear(in_features=10, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=1, bias=True)


  0%|          | 0/2000 [00:00<?, ?it/s]c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([84])) that is different to the input size (torch.Size([84, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 2000/2000 [01:07<00:00, 29.68it/s]

Validation Loss for fold 5: 0.9734272062778473
Final CV Loss: 1.0323131561279297
